In [27]:
import os
import prody as pr
import networkx as nx
import numpy as np
import requests
import gzip
import shutil
import pandas as pd
from scipy.spatial.distance import cdist
import math
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import plotly.graph_objects as go
from Bio import PDB
import py3Dmol
from pyvis.network import Network
from IPython.core.display import display, HTML

# Function to fetch molecular weight from PubChem API
def get_molecular_weight(ligand_id):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{ligand_id}/property/MolecularWeight/JSON"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return float(data["PropertyTable"]["Properties"][0]["MolecularWeight"])
    except:
        # Adaptive default based on ligand size
        if len(ligand_id) <= 3:  # Likely a small ion (e.g., Na+, Cl-)
            return 50.0
        else:  # Small organic molecule
            return 180.0  

def download_and_extract_cif(pdb_id):
    pdb_id = pdb_id.upper().strip()
    url = f"https://files.rcsb.org/download/{pdb_id}-assembly1.cif.gz"
    cif_gz_file = f"{pdb_id}-assembly1.cif.gz"
    cif_file = f"{pdb_id}-assembly1.cif"

    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error: No Biological Assembly CIF file found for {pdb_id} on RCSB PDB.")
        return None
    
    with open(cif_gz_file, "wb") as f:
        f.write(response.content)
    print(f"Downloaded CIF file: {cif_gz_file}")

    with gzip.open(cif_gz_file, "rb") as f_in, open(cif_file, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
    print(f"Extracted CIF file: {cif_file}")

    os.remove(cif_gz_file)
    return cif_file

def convert_cif_to_pdb(cif_file, pdb_output):
    structure = pr.parseMMCIF(cif_file)
    pr.writePDB(pdb_output, structure)
    print(f"Converted {cif_file} → {pdb_output}")
    return pdb_output

def get_ligand_residues(pdb_file):
    """Extracts ligand residue names from a PDB file (excluding proteins & water)."""
    structure = pr.parsePDB(pdb_file)
    ligand_atoms = structure.select("not protein and not water")
    
    if ligand_atoms is None:
        return []
    
    return list(set(ligand_atoms.getResnames()))  # Get unique ligand names

def visualize_pdb(pdb_file, highlight_residues=None):
    """Visualizes protein structure with ligands and highlights a specific residue."""
    if not os.path.exists(pdb_file):
        print(f"Error: File {pdb_file} does not exist.")
        return
    
    with open(pdb_file, "r") as f:
        pdb_content = f.read()
    
    view = py3Dmol.view(width=800, height=600)
    view.addModel(pdb_content, "pdb")
    view.setStyle({"cartoon": {"color": "spectrum"}})

    # Highlight ligand residues
    ligand_residues = get_ligand_residues(pdb_file)
    if ligand_residues:
        for ligand in ligand_residues:
            view.addStyle({"resn": ligand, "hetflag": True}, 
                          {"stick": {"colorscheme": "greenCarbon", "radius": 0.3}})

        # Highlight Multiple Residues with Different Colors
    if highlight_residues:
        for res in highlight_residues:
            chain, resname, resnum, color = res  # Example: ("A", "ASP", 50, "red")
            view.addStyle(
                {"chain": chain, "resn": resname, "resi": str(resnum)},
                {"sphere": {"color": color, "radius": 1.5}}
            )
    
    view.zoomTo()
    return view.show()

# Example Usage:
# visualize_pdb("example.pdb", highlight_residue=("A", "ASP", 50))  # Highlights ASP 50 in Chain A


def process_pdb_id(pdb_id, save_directory="."):
    cif_file = download_and_extract_cif(pdb_id)
    if cif_file:
        pdb_output = os.path.join(save_directory, f"{pdb_id}.pdb")
        pdb_file = convert_cif_to_pdb(cif_file, pdb_output)
        #visualize_pdb(pdb_file)
        highlight_residues = [
            ("A", "VAL", 122, "green"),   # Highlight ASP 50 in red
            ("A", "ASP", 170, "red"), # Highlight LYS 120 in blue
        ]
        visualize_pdb(pdb_file, highlight_residues=highlight_residues)
        analyze_protein_network(pdb_file, pdb_id)



# Function to detect ligand-protein hydrogen bonds
def detect_hydrogen_bonds(protein_atoms, ligand_atoms, distance_cutoff=3.5):
    """Detects hydrogen bonds only between ligand and protein atoms."""
    hbond_pairs = []

    if protein_atoms is None or ligand_atoms is None:
        print("Warning: No valid protein or ligand atoms found. Skipping H-bond detection.")
        return hbond_pairs  # Return empty list

    protein_atoms_filtered = protein_atoms.select("element N O")
    ligand_atoms_filtered = ligand_atoms.select("element N O")

    if protein_atoms_filtered is None or ligand_atoms_filtered is None:
        print("Warning: No donor/acceptor atoms found in protein or ligand.")
        return hbond_pairs  # Return empty list

    for ligand_atom in ligand_atoms_filtered:
        for protein_atom in protein_atoms_filtered:
            distance = pr.calcDistance(ligand_atom, protein_atom)
            if distance <= distance_cutoff:
                ligand_res_id = f"{ligand_atom.getChid()}-{ligand_atom.getResname()}{ligand_atom.getResnum()}"
                protein_res_id = f"{protein_atom.getChid()}-{protein_atom.getResname()}{protein_atom.getResnum()}"
                hbond_pairs.append((ligand_res_id, protein_res_id))

    return hbond_pairs[:200]  # Limit max bonds to 200 for efficiency

# Function to analyze protein-ligand network
def analyze_protein_network(pdb_file, pdb_id, distance_cutoff=7.0, network_type="CA"):
    structure = pr.parsePDB(pdb_file)
    
    protein_atoms = structure.select("name CA") if network_type == "CA" else structure.select("name CB")
    ligand_atoms = structure.select("not protein and not water")

    if protein_atoms is None:
        print(" Warning: No protein atoms found in structure.")
    if ligand_atoms is None:
        print(" Warning: No ligand atoms found in structure.")

    G = nx.Graph()
    node_positions = {}

    # 🔹 Add Protein Nodes & Edges
    if protein_atoms is not None:
        for res in protein_atoms:
            res_id = f"{res.getChid()}-{res.getResname()}{res.getResnum()}"
            coords = res.getCoords()
            node_positions[res_id] = coords
            G.add_node(res_id)

        for i, res1 in enumerate(protein_atoms):
            for j, res2 in enumerate(protein_atoms):
                if i < j:
                    distance = pr.calcDistance(res1, res2)
                    if distance < distance_cutoff:
                        res1_id = f"{res1.getChid()}-{res1.getResname()}{res1.getResnum()}"
                        res2_id = f"{res2.getChid()}-{res2.getResname()}{res2.getResnum()}"
                        G.add_edge(res1_id, res2_id, weight=2.0)

    # Normalize Protein-Protein Edge Weights
    if len(G.edges) > 0:
        avg_weight = np.mean([d["weight"] for u, v, d in G.edges(data=True)])
        for u, v, d in G.edges(data=True):
            d["weight"] /= avg_weight * 10.0  

    # Add Ligand Nodes & Edges using Hydrogen Bonds
    if ligand_atoms is not None and protein_atoms is not None:
        hbond_pairs = detect_hydrogen_bonds(protein_atoms, ligand_atoms)

        for ligand_res, protein_res in hbond_pairs:
            if ligand_res in G.nodes and protein_res in G.nodes:
                G.add_edge(ligand_res, protein_res, weight=1.0)


    # Step 3: Compute Centrality Measures
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G, weight="weight")
    closeness_centrality = nx.closeness_centrality(G)
    eigenvector_centrality = nx.eigenvector_centrality_numpy(G, weight="weight")
    pagerank_centrality = nx.pagerank(G, alpha=0.85)

    # Compute Network Centrality Measures
    centralities = {
        "degree": nx.degree_centrality(G),
        "betweenness_centrality": nx.betweenness_centrality(G, weight="weight"),
        "closeness_centrality": nx.closeness_centrality(G),
        "eigenvector_centrality": nx.eigenvector_centrality_numpy(G, weight="weight"),
        "pageRank_centrality": nx.pagerank(G, alpha=0.85)
    }

    # Save to Excel
    df = pd.DataFrame([[node] + [centralities[m][node] for m in centralities] for node in G.nodes],
                      columns=["Node"] + list(centralities.keys()))
    df.to_excel(f"{pdb_id}_network_centrality_with_ligands.xlsx", index=False)

    print(f"Network centrality measures saved to {pdb_id}_network_centrality_with_ligands.xlsx")

    plot_network_3d(G, betweenness_centrality, "Betweenness Centrality", node_positions)
    plot_network_pyvis(G, betweenness_centrality, "Betweenness Centrality")
    #plot_network_pyvis(G, betweenness_centrality, "Betweenness Centrality")

def plot_network_3d(G, centrality_measure, centrality_name, node_positions):
    node_x, node_y, node_z, node_color, node_text = [], [], [], [], []
    for node, coords in node_positions.items():
        node_x.append(coords[0])
        node_y.append(coords[1])
        node_z.append(coords[2])
        node_color.append(centrality_measure.get(node, 0))
        node_text.append(f"Residue: {node}<br>{centrality_name}: {centrality_measure.get(node, 0):.4f}")
    
    edge_x, edge_y, edge_z = [], [], []
    for edge in G.edges():
        res1, res2 = edge
        x0, y0, z0 = node_positions[res1]
        x1, y1, z1 = node_positions[res2]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])
    
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='gray', width=2),
        hoverinfo='none',
        showlegend=False
    ))
    fig.add_trace(go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers',
        marker=dict(
            size=8,
            color=node_color,
            colorscale='Plasma',
            colorbar=dict(title=centrality_name, len=0.5),
            opacity=0.8
        ),
        text=node_text,
        hoverinfo='text',
        showlegend=False
    ))
    
    fig.update_layout(
        title=f'Protein Residue Interaction Network ({centrality_name})',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        ),
        margin=dict(l=0, r=0, b=0, t=80)
    )
    
    fig.show()

def plot_network_pyvis(G, centrality_measure, centrality_name):
    """Plots an interactive Pyvis network visualization with ligands explicitly shown."""

    net = Network(notebook=True, height="600px", width="1000px", bgcolor="white", font_color="black", directed=False)

    # Define the colormap and normalization
    cmap = plt.get_cmap("coolwarm")
    norm = mcolors.Normalize(vmin=min(centrality_measure.values()), vmax=max(centrality_measure.values()))

    # Add nodes to the network with distinct colors and bold labels for ligands
    for node in G.nodes():
        if node.startswith("LIG-"):  
            color = "green"  # Ligands are explicitly green
            size = 30  # Make ligands larger
            font_size = 20  # Increase font size for ligand labels
            font_style = {"size": font_size, "color": "black", "bold": True}  # Apply Pyvis bold
        else:
            color = mcolors.to_hex(cmap(norm(centrality_measure[node])))
            size = 15  # Default size for protein residues
            font_style = {"size": 12, "color": "black", "bold": False}  # Normal font

        # Add node with styling
        net.add_node(
            node, 
            label=f"{node} ({centrality_measure.get(node, 0):.4f})", 
            size=size, 
            color=color, 
            font=font_style
        )

    # Add edges to the network
    for edge in G.edges():
        net.add_edge(edge[0], edge[1])

    # Generate and display the network
    net.show("protein_network.html")
    display(HTML("<iframe src='protein_network.html' width='1000px' height='600px'></iframe>"))


pdb_id = "3ZHB"
process_pdb_id(pdb_id)


/var/folders/77/wfv9nbcd5t94hwcgvb3vzvwr0000gn/T/ipykernel_20741/3486484372.py:17: DeprecationWarning:

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display

@> 4161 atoms and 1 coordinate set(s) were parsed in 0.06s.
@> 4161 atoms and 1 coordinate set(s) were parsed in 0.03s.


Downloaded CIF file: 3ZHB-assembly1.cif.gz
Extracted CIF file: 3ZHB-assembly1.cif
Converted 3ZHB-assembly1.cif → ./3ZHB.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

@> 4161 atoms and 1 coordinate set(s) were parsed in 0.03s.


Network centrality measures saved to 3ZHB_network_centrality_with_ligands.xlsx


protein_network.html


/opt/miniconda3/lib/python3.12/site-packages/IPython/core/display.py:431: UserWarning:

Consider using IPython.display.IFrame instead



In [41]:
import os
import prody as pr
import networkx as nx
import numpy as np
import pandas as pd
import requests
from IPython.display import display
import py3Dmol

# Download PDB from RCSB
def process_pdb_id(pdb_id, save_directory="."):
    pdb_id = pdb_id.upper().strip()
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    pdb_file = os.path.join(save_directory, f"{pdb_id}.pdb")
    with open(pdb_file, "w") as f:
        f.write(response.text)
    return pdb_file

# Ligand-contact residues
def extract_ligand_contact_residues(pdb_file, distance_cutoff=4.0):
    structure = pr.parsePDB(pdb_file)
    protein_atoms = structure.select("protein and name CA")
    ligand_atoms = structure.select("not protein and not water")
    if protein_atoms is None or ligand_atoms is None:
        return []
    contact_residues = set()
    for ligand_atom in ligand_atoms:
        for protein_atom in protein_atoms:
            if pr.calcDistance(ligand_atom, protein_atom) <= distance_cutoff:
                resname = protein_atom.getResname()
                resnum = protein_atom.getResnum()
                chain = protein_atom.getChid()
                contact_residues.add(f"{chain}-{resname}{resnum}")
    return list(contact_residues)

# PRS and memory computation
def compute_memory_condition_prs(pdb_file, ligand_contact_residues, distance_cutoff=7.0):
    structure = pr.parsePDB(pdb_file)
    calphas = structure.select('protein and name CA')
    if calphas is None or calphas.numAtoms() < 2:
        raise ValueError("Not enough C-alpha atoms found.")

    anm = pr.ANM('ANM analysis')
    anm.buildHessian(calphas, cutoff=distance_cutoff)
    anm.calcModes(n_modes=20)
    prs_matrix = pr.calcPerturbResponse(anm)[0]

    G = nx.Graph()
    coords = calphas.getCoords()
    res_ids = [f"{res.getChid()}-{res.getResname()}{res.getResnum()}" for res in calphas]

    for i, res_id in enumerate(res_ids):
        G.add_node(res_id, coords=coords[i])
    for i in range(len(coords)):
        for j in range(i + 1, len(coords)):
            dist = np.linalg.norm(coords[i] - coords[j])
            if dist <= distance_cutoff:
                k_ij = 1.0 / dist**2
                q_ij = prs_matrix[i, j]
                memory_score = q_ij**2 / k_ij
                G.add_edge(res_ids[i], res_ids[j], weight=memory_score, q_ij=q_ij, k_ij=k_ij)

    gamma_threshold = np.percentile([d["weight"] for _, _, d in G.edges(data=True)], 95)
    retained_edges = [(u, v, d["q_ij"], d["k_ij"], d["weight"]) 
                      for u, v, d in G.edges(data=True) if d["weight"] > gamma_threshold]

    return {
        "graph": G,
        "gamma_threshold": gamma_threshold,
        "retained_edges": retained_edges
    }

# Visualize in Py3Dmol
def visualize_pdb_with_pathways(pdb_file, memory_edges, ligand_color='green', pathway_color='red'):
    formatted = []
    for item in memory_edges:
        try:
            res1, res2, qij, kij, score = item
        except ValueError:
            res1, res2, attrs = item
            qij = attrs.get("q_ij")
            kij = attrs.get("k_ij")
            score = attrs.get("weight")
        formatted.append((res1, res2, qij, kij, score))

    df = pd.DataFrame(formatted, columns=["Residue_i", "Residue_j", "q_ij", "k_ij", "q²/k"])
    gamma_threshold = df["q²/k"].quantile(0.95)
    top_edges = df[df["q²/k"] > gamma_threshold]
    pathway_residues = set(top_edges["Residue_i"]).union(set(top_edges["Residue_j"]))

    # Load full PDB data
    with open(pdb_file, 'r') as f:
        pdb_data = f.read()

    view = py3Dmol.view(width=1000, height=700)
    view.addModel(pdb_data, "pdb")

    # Remove all default styles
    view.setStyle({}, {})

    # Show chains A and B only
    for chain in ["A", "B"]:
        view.setStyle({"chain": chain, "model": 0}, {"cartoon": {"color": "spectrum"}})

    # Load ligands only for chains A and B
    structure = pr.parsePDB(pdb_file)
    ligand_atoms = structure.select("not protein and not water and (chain A or chain B)")

    if ligand_atoms:
        for lig in set(ligand_atoms.getResnames()):
            view.addStyle(
                {"resn": lig, "hetflag": True, "chain": ["A", "B"], "model": 0},
                {"stick": {"colorscheme": ligand_color + "Carbon", "radius": 0.3}}
            )

    # Highlight memory residues (spheres)
    for res_id in pathway_residues:
        try:
            chain, rest = res_id.split("-")
            resname = ''.join([c for c in rest if not c.isdigit()])
            resnum = ''.join([c for c in rest if c.isdigit()])
            if chain in ["A", "B"]:
                view.addStyle(
                    {"chain": chain, "resn": resname, "resi": resnum, "model": 0},
                    {"sphere": {"color": pathway_color, "radius": 1.5}}
                )
        except Exception as e:
            print(f"Skipping {res_id}: {e}")

    view.zoomTo()
    view.show()

    # Print top memory scores
    display(df[df["q²/k"] > gamma_threshold].sort_values("q²/k", ascending=False).reset_index(drop=True))

# === Run Pipeline ===
pdb_id = "3ZHB"
pdb_file = process_pdb_id(pdb_id)
ligand_residues = extract_ligand_contact_residues(pdb_file)
results = compute_memory_condition_prs(pdb_file, ligand_residues)
visualize_pdb_with_pathways(pdb_file, results["retained_edges"])


@> 8395 atoms and 1 coordinate set(s) were parsed in 0.06s.
@> 8395 atoms and 1 coordinate set(s) were parsed in 0.06s.
@> Hessian was built in 0.11s.
@> WARNING More than 6 (7) zero eigenvalues were calculated.
@> Solving for additional eigenvalues...
@> 20 modes were calculated in 10.17s.
@> 8395 atoms and 1 coordinate set(s) were parsed in 0.06s.


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

,Residue_i,Residue_j,q_ij,k_ij,q²/k
0,B-ALA138,B-PRO140,2.719220,0.021840,338.566889
1,B-LEU151,B-SER153,2.473702,0.020855,293.412255
2,A-VAL229,A-ASP231,3.114839,0.035474,273.502205
3,B-PRO276,B-LYS280,2.537078,0.024666,260.956498
4,B-LEU152,B-GLY154,2.263497,0.020946,244.599853
5,A-VAL229,A-TRP232,2.874333,0.035383,233.494237
6,B-ILE279,B-ALA283,2.279193,0.022437,231.521502
7,A-ALA219,B-ALA237,2.166794,0.021687,216.484976
8,B-THR51,B-ALA53,2.211514,0.024688,198.103970
9,B-GLY204,B-LEU208,2.226818,0.025813,192.098458


In [57]:
import os
import gzip
import shutil
import prody as pr
import networkx as nx
import numpy as np
import pandas as pd
import requests
from IPython.display import display
import py3Dmol

# === Download CIF & Convert to PDB ===
def download_and_extract_cif(pdb_id):
    pdb_id = pdb_id.upper().strip()
    url = f"https://files.rcsb.org/download/{pdb_id}-assembly1.cif.gz"
    cif_gz_file = f"{pdb_id}-assembly1.cif.gz"
    cif_file = f"{pdb_id}-assembly1.cif"

    response = requests.get(url)
    if response.status_code != 200:
        print(f"No biological assembly CIF file found for {pdb_id}, falling back to PDB.")
        return None

    with open(cif_gz_file, "wb") as f:
        f.write(response.content)
    with gzip.open(cif_gz_file, "rb") as f_in, open(cif_file, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
    os.remove(cif_gz_file)
    return cif_file

def convert_cif_to_pdb(cif_file, pdb_output):
    structure = pr.parseMMCIF(cif_file)
    pr.writePDB(pdb_output, structure)
    return pdb_output

# === PDB Processing ===
def process_pdb_id(pdb_id, save_directory="."):
    cif_file = download_and_extract_cif(pdb_id)
    pdb_file = os.path.join(save_directory, f"{pdb_id}.pdb")

    if cif_file:
        pdb_file = convert_cif_to_pdb(cif_file, pdb_file)
    else:
        url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
        response = requests.get(url)
        with open(pdb_file, "w") as f:
            f.write(response.text)
    return pdb_file

# === Chain Detection ===
def detect_all_chains(pdb_file):
    structure = pr.parsePDB(pdb_file)
    return sorted(set(atom.getChid() for atom in structure if atom.getChid().strip()))

# === Ligand-contact residues ===
def extract_ligand_contact_residues(pdb_file, distance_cutoff=4.0):
    structure = pr.parsePDB(pdb_file)
    protein_atoms = structure.select("protein and name CA")
    ligand_atoms = structure.select("not protein and not water")
    contact_residues = set()
    if protein_atoms and ligand_atoms:
        for ligand_atom in ligand_atoms:
            for protein_atom in protein_atoms:
                if pr.calcDistance(ligand_atom, protein_atom) <= distance_cutoff:
                    resname = protein_atom.getResname()
                    resnum = protein_atom.getResnum()
                    chain = protein_atom.getChid()
                    contact_residues.add(f"{chain}-{resname}{resnum}")
    return list(contact_residues)

# === PRS Computation ===
def compute_memory_condition_prs(pdb_file, ligand_contact_residues, distance_cutoff=7.0):
    structure = pr.parsePDB(pdb_file)
    calphas = structure.select('protein and name CA')
    anm = pr.ANM('ANM analysis')
    anm.buildHessian(calphas, cutoff=distance_cutoff)
    anm.calcModes(n_modes=20)
    prs_matrix = pr.calcPerturbResponse(anm)[0]

    G = nx.Graph()
    coords = calphas.getCoords()
    res_ids = [f"{res.getChid()}-{res.getResname()}{res.getResnum()}" for res in calphas]

    for i, res_id in enumerate(res_ids):
        G.add_node(res_id, coords=coords[i])
    for i in range(len(coords)):
        for j in range(i + 1, len(coords)):
            dist = np.linalg.norm(coords[i] - coords[j])
            if dist <= distance_cutoff:
                k_ij = 1.0 / dist**2
                q_ij = prs_matrix[i, j]
                memory_score = q_ij**2 / k_ij
                G.add_edge(res_ids[i], res_ids[j], weight=memory_score, q_ij=q_ij, k_ij=k_ij)

    gamma_threshold = np.percentile([d["weight"] for _, _, d in G.edges(data=True)], 95)
    retained_edges = [(u, v, d["q_ij"], d["k_ij"], d["weight"]) 
                      for u, v, d in G.edges(data=True) if d["weight"] > gamma_threshold]

    centralities = {
        "betweenness": nx.betweenness_centrality(G, weight="weight"),
        "closeness": nx.closeness_centrality(G)
    }

    return {
        "graph": G,
        "gamma_threshold": gamma_threshold,
        "retained_edges": retained_edges,
        "centralities": centralities
    }

# === Visualization ===
def visualize_pdb_with_pathways(pdb_file, memory_edges, centralities, ligand_color='green'):
    structure = pr.parsePDB(pdb_file)
    all_chains = detect_all_chains(pdb_file)

    df = pd.DataFrame([
        (res1, res2, qij, kij, score)
        for res1, res2, qij, kij, score in memory_edges
    ], columns=["Residue_i", "Residue_j", "q_ij", "k_ij", "q²/k"])

    gamma_threshold = df["q²/k"].quantile(0.95)
    top_edges = df[df["q²/k"] > gamma_threshold]
    memory_residues = set(top_edges["Residue_i"]).union(set(top_edges["Residue_j"]))

    valid_residues = {
        f"{res.getChid()}-{res.getResname()}{res.getResnum()}"
        for res in structure.select(f"protein and chain {' '.join(all_chains)}")
    }

    top_labels = {}
    colors = {"betweenness": "green", "closeness": "blue"}

    for centrality, values in centralities.items():
        top5 = sorted(values.items(), key=lambda x: x[1], reverse=True)
        count = 0
        for res, score in top5:
            if res in valid_residues:
                top_labels.setdefault(res, {"color": colors[centrality], "tooltip": []})
                top_labels[res]["tooltip"].append(f"{centrality}: {score:.3f}")
                count += 1
            if count >= 5:
                break

    all_residues = memory_residues.union(set(top_labels.keys())).intersection(valid_residues)

    with open(pdb_file, 'r') as f:
        pdb_data = f.read()

    view = py3Dmol.view(width=1000, height=700)
    view.addModel(pdb_data, "pdb")
    view.setStyle({}, {})

    for chain in all_chains:
        view.setStyle({"chain": chain, "model": 0}, {"cartoon": {"color": "spectrum"}})

    ligand_atoms = structure.select(f"not protein and not water and chain {' '.join(all_chains)}")
    if ligand_atoms:
        for lig in set(ligand_atoms.getResnames()):
            view.addStyle({"resn": lig, "hetflag": True},
                          {"stick": {"colorscheme": ligand_color + "Carbon", "radius": 0.3}})

    for res_id in all_residues:
        try:
            chain, rest = res_id.split('-')
            resname = ''.join([c for c in rest if not c.isdigit()])
            resnum = ''.join([c for c in rest if c.isdigit()])
            color = top_labels.get(res_id, {}).get("color", "red")
            tooltip = "<br>".join(top_labels.get(res_id, {}).get("tooltip", []))
            radius = 1.5 if res_id in top_labels else 1.0
            view.addStyle({"chain": chain, "resn": resname, "resi": resnum, "model": 0},
                          {"sphere": {"color": color, "radius": radius}})
            if tooltip:
                view.addLabel(tooltip, {"position": {"resi": int(resnum), "chain": chain},
                                        "backgroundColor": "white",
                                        "fontColor": color,
                                        "fontSize": 10})
        except Exception as e:
            print(f"Skipping {res_id}: {e}")

    view.zoomTo()
    view.show()

    print("\nTop 5 Memory Edges (q²/k):")
    display(df[df["q²/k"] > gamma_threshold].sort_values("q²/k", ascending=False).reset_index(drop=True).head(5))

    for key, val in centralities.items():
        print(f"\nTop 5 Centralities ({key}):")
        display(pd.Series(val).sort_values(ascending=False).head(5))

# === Run Full Pipeline ===
pdb_id = "3ZHB"
pdb_file = process_pdb_id(pdb_id)
ligand_residues = extract_ligand_contact_residues(pdb_file)
results = compute_memory_condition_prs(pdb_file, ligand_residues)
visualize_pdb_with_pathways(pdb_file, results["retained_edges"], results["centralities"])


@> 4161 atoms and 1 coordinate set(s) were parsed in 0.07s.
@> 4161 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 4161 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> Hessian was built in 0.04s.
@> WARNING More than 6 (7) zero eigenvalues were calculated.
@> Solving for additional eigenvalues...
@> 20 modes were calculated in 1.13s.
@> 4161 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 4161 atoms and 1 coordinate set(s) were parsed in 0.03s.


3Dmol.js failed to load for some reason. Please check your browser console for error messages.


Top 5 Memory Edges (q²/k):


,Residue_i,Residue_j,q_ij,k_ij,q²/k
0,A-ALA135,A-LEU137,3.386468,0.023728,483.327707
1,B-ALA135,B-LEU137,3.215878,0.023711,436.159058
2,B-LEU132,B-ALA158,2.454209,0.022150,271.924260
3,A-LEU152,A-THR174,2.633378,0.026669,260.026951
4,B-LEU152,B-THR174,2.594609,0.026644,252.662979



Top 5 Centralities (betweenness):


B-ASP133    0.256610
B-ALA117    0.253126
B-LEU185    0.240683
B-HIS261    0.226495
A-GLY112    0.225117
dtype: float64


Top 5 Centralities (closeness):


A-MET193    0.166376
B-MET193    0.166086
B-ILE197    0.165414
A-ILE197    0.165414
A-MET194    0.165366
dtype: float64